# Chapter 12-04
## 일본 인구를 시각화한다

### 인구 데이터를 읽어 들임

In [ ]:
# 1944년부터 2014년까지의 5계층 단계별로 인구를 남녀별로 읽어 들임
import numpy as np

p_male = np.loadtxt('male_1944_2014.csv', delimiter=",",
                      skiprows=1, usecols=range(1, 22))
p_female = np.loadtxt('female_1944_2014.csv', delimiter=",",
                      skiprows=1, usecols=range(1, 22))

In [ ]:
# 계급별과 연도별로 인구를 계산한다.
# 남녀 5세씩의 단계별로 인구를 합계
p_total = p_male+p_female
# 연도별 인구로 수정
p_yearly = p_total.sum(axis=1)

In [ ]:
# 연도별 인구를 그래프화 한다
%matplotlib inline
import matplotlib.pyplot as plt

t = plt.plot(range(1944, 2015), p_yearly)
plt.ylim((0, 130000))
plt.grid(True)

In [ ]:
# 출생률(합계 특수 출생률) 그래프를 그린다
# 1960년에서 2014년을 읽어 들임
tfr = np.loadtxt('total_fertility_rate.csv', delimiter=",", skiprows=1)
# 출산율 그래프를 그린다
t = plt.plot(range(1960, 2015), tfr, ls=":")
# 인구 치환 경계선을 2.07로 하여 선을 그림
t = plt.plot([1960, 2015], [2.07, 2.07])

### 인구 피라미드를 그린다

In [ ]:
# show_pgraph( ) 함수의 정의
from matplotlib import gridspec
def show_pgraph(year, arr1, arr2, arr3,
                ymin, ymax, ydim=1):
    # 인구 피라미드를 표시
    # 표시하는 인구의 인덱스를 가져옴
    idx = int((year-ymin)/ydim)
    # 인구 피라미드와 인구 그래프의 그리드를 생성
    gs = gridspec.GridSpec(2, 2, height_ratios=(3, 2))
    # 그래프의 배치를 결정
    ax = [plt.subplot(gs[0, 0]),plt.subplot(gs[0, 1]),
          plt.subplot(gs[1, :])]
    # 남성 인구 피라미드를 그림
    ax[0].barh(range(0, 101, 5), arr1[idx], height=3)
    ax[0].set(ylim=(0, 100), xlim=(0, 6000))
    ax[0].invert_xaxis()
    ax[0].yaxis.tick_right()
    # 여성 인구 피라미드를 그림
    ax[1].barh(range(0, 101, 5), arr2[idx], height=3)
    ax[1].tick_params(labelleft='off')
    ax[1].set(ylim=(0, 100), xlim=(0, 6000))
    # 인구 그래프를 그림
    ax[2].plot(range(ymin, ymax+1, ydim), arr3, ls=":")
    ax[2].plot([year, year], [0, 140000])

In [ ]:
# 인구 피라미드의 그래프를 그린다
show_pgraph(1950, p_male, p_female, p_yearly,
            1944, 2014)

### 그래프를 인터랙티브(interactive)하게 그린다

In [ ]:
# 슬라이더를 사용하여 그래프를 그린다
from ipywidgets import interact, IntSlider, fixed

t = interact(show_pgraph, year=IntSlider(min=1944, max=2014, step=5),
         arr1=fixed(p_male), arr2=fixed(p_female), arr3=fixed(p_yearly),
         ymin=fixed(1944), ymax=fixed(2014), ydim=fixed(1))

### 장래 인구를 추계(일부를 가지고 전체를 미루어 계산하는 것)한다

In [ ]:
# 각 데이터를 배열에 사용
# 추정에 필요한 데이터를 읽어 들임
# 2014년 사망률(5세 단계별)을 읽어 들임
lifechart = np.loadtxt('lifechart2014.csv', delimiter=",", usecols=[3])
# 사망률을 1에서 뺀 array로 만듦
rev_lifechart = np.ones(lifechart.size)-lifechart
# 인구 대체 수준 상당의 출생률(15세부터 5년마다)
rep_level = np.array([0.0041, 0.107,
        0.19, 0.0697, 0.017, 0.0021, 0.0001])
# 고위 추계에 상당하는 출생률
high_rate = np.array([0.0036, 0.0514,
        0.1593, 0.0927, 0.0187, 0.0023, 0.0001])

In [ ]:
# 인구 추계 처리
# 장래 인구 추계를 처리
# 남녀 인구 데이터를 추정하는 array를 정의
fp_male = np.array(p_male[-2:])
fp_female = np.array(p_female[-2:])

# 인구 치환 수준으로 돌아가는 기간(5로 나눔)
recover_in = 5

for i in range(20):
    # 100년간 분을 5년마다 반복
    # 최근의 5세 계급별로 새로운 인구를 초기화
    new_fp_male = fp_male[-1]
    new_fp_female = fp_female[-1]
    # 출생률을 설정
    if i > recover_in:
        f_rate = rep_level
    else:
        f_rate = high_rate
    # 15-49세의 여성 인구에 출생률을 곱하여 신생아 수를 계산
    newborn = np.sum(new_fp_female[3:10]*f_rate)*5
    # 최근의 5세 계급별 인구 인덱스를
    # 오른쪽으로 이동하여 신생아를 왼쪽에 연결
    new_fp_male = np.hstack(
        ([newborn*0.4878], new_fp_male[:-1]))
    new_fp_female = np.hstack(
        ([newborn*0.5122], new_fp_female[:-1]))
    # 각 단계의 인구에 사망률을 적용
    new_fp_male *= rev_lifechart
    new_fp_female *= rev_lifechart
    # 새로운 추정 인구를 추가
    fp_male = np.vstack(
        (fp_male, new_fp_male))
    fp_female = np.vstack(
        (fp_female, new_fp_female))

# 남녀 합산의 5세 단계별 인구, 5년마다의 추정 총인구 array를 작성
fp_total = fp_male+fp_female
fp_sum = np.array([np.sum(x) for x in fp_total])

In [ ]:
# 그래프의 표시
# 2120년까지의 인구 추계 그래프를 그린다
t=plt.plot(range(2013, 2120, 5), fp_sum)
t=plt.ylim([0, 130000])
plt.grid(True)

In [ ]:
# 인구 추계를 그래프로 그린다
# 인구 추계 데이터를 슬라이더에 동적으로 표시
t = interact(show_pgraph, year=IntSlider(min=2013, max=2113, step=5),
         arr1=fixed(fp_male), arr2=fixed(fp_female), arr3=fixed(fp_sum),
         ymin=fixed(2013), ymax=fixed(2120), ydim=fixed(5))